# Env

In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,accuracy_score,explained_variance_score
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from xgboost import plot_importance
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.base import clone
from sklearn.inspection import permutation_importance
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from bayes_opt import BayesianOptimization
from sklearn.metrics import make_scorer
from copy import deepcopy


# MLP

In [ ]:
class RobustNN(nn.Module):
    def __init__(self, input_dim):
        super(RobustNN, self).__init__()
        self.feature_weights = nn.Parameter(torch.ones(input_dim)) 
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        x = x * self.feature_weights
        return self.net(x)

def loss_fn_with_l1(output, target, model, l1_lambda=0.01):
    mse_loss = nn.MSELoss()(output, target)
    l1_penalty = l1_lambda * torch.norm(model.feature_weights, p=1)
    return mse_loss + l1_penalty

def train_model(model, train_loader, val_loader, num_epochs=1000, patience=60, l1_lambda=0.01, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None
    train_loss_history = []
    val_loss_history = []

    for epoch in range(num_epochs):
        model.train()
        train_losses = []

        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            preds = model(batch_X).squeeze()
            loss = loss_fn_with_l1(preds, batch_y, model, l1_lambda)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        model.eval()
        val_losses = []
        with torch.no_grad():
            for val_X, val_y in val_loader:
                val_preds = model(val_X).squeeze()
                val_loss = loss_fn_with_l1(val_preds, val_y, model, l1_lambda)
                val_losses.append(val_loss.item())

        avg_train_loss = np.mean(train_losses)
        avg_val_loss = np.mean(val_losses)
        train_loss_history.append(avg_train_loss)
        val_loss_history.append(avg_val_loss)

        # print(f"Epoch {epoch+1:03d} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                # print("Early stopping triggered!")
                break

    model.load_state_dict(best_model_state)

    # 可视化 loss 曲线
    # plt.plot(train_loss_history, label='Train Loss')
    # plt.plot(val_loss_history, label='Val Loss')
    # plt.xlabel('Epoch')
    # plt.ylabel('Loss')
    # plt.legend()
    # plt.title("Training and Validation Loss")
    # plt.grid(True)
    # plt.show()

    return model

def evaluate_model(model, test_loader, y_scaler):
    model.eval()
    predictions = []
    targets = []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            output = model(X_batch).squeeze()
            predictions.append(output.cpu().numpy())
            targets.append(y_batch.cpu().numpy())

    preds = np.concatenate(predictions)
    trues = np.concatenate(targets)

    preds_real = y_scaler.inverse_transform(preds.reshape(-1, 1)).ravel()
    trues_real = y_scaler.inverse_transform(trues.reshape(-1, 1)).ravel()

    mae = mean_absolute_error(trues_real, preds_real)
    mse = mean_squared_error(trues_real, preds_real)
    rmse = np.sqrt(mse)
    r2 = r2_score(trues_real, preds_real)


    # print("\n=== 测试集评估结果 ===")
    # print(f"MAE: {mae:.4f}")
    # print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2: {r2:.4f}")

    return preds_real, trues_real 

# Data Loading

In [4]:
df = pd.read_excel("generated_samples_robust.xlsx")
df.describe()

,Fe,Cr,Ni,Mn,Al,Cu,Co,Epit
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.203058,0.236246,0.239013,0.003701,0.025238,0.066468,0.226276,356.611982
std,0.042621,0.028618,0.029026,0.004025,0.022540,0.054250,0.035668,324.940857
min,0.077745,0.131589,0.139919,0.000040,0.000039,0.000538,0.141893,-405.018555
25%,0.174752,0.220172,0.220218,0.001064,0.005027,0.018572,0.200640,106.186977
50%,0.204421,0.239786,0.239522,0.002220,0.018842,0.051741,0.228763,423.067719
75%,0.233464,0.255418,0.260141,0.004766,0.041972,0.107491,0.255375,626.818359
max,0.306684,0.306073,0.311860,0.030913,0.087726,0.192010,0.301496,861.663879


In [97]:
X = df.drop(columns=['Epit'])
y = df['Epit']

In [198]:
df = pd.read_excel("generated_samples_robust_feature.xlsx")
df.describe()

,Fe,Cr,Ni,Mn,Al,Cu,Co,E_M-M,Gibbs free energy of oxide formation_Aver,Energy of ionization second_Aver,Epit
count,500.000000,500.000000,500.000000,5.000000e+02,5.000000e+02,5.000000e+02,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.223816,0.227380,0.228819,3.304442e-02,1.863189e-02,4.050260e-02,0.227805,81.367155,-88.989191,1651.587129,232.402195
std,0.049817,0.030970,0.036191,6.694506e-02,3.530871e-02,6.545965e-02,0.034336,3.386451,30.019819,24.654049,280.525059
min,0.080707,0.116132,0.070408,5.789687e-09,1.006968e-07,5.511108e-12,0.099182,71.424179,-168.027863,1598.270264,-374.637360
25%,0.197601,0.209846,0.208479,7.257671e-05,1.019616e-04,3.654962e-06,0.206515,79.227253,-100.980476,1637.834961,43.151689
50%,0.223239,0.229495,0.235907,1.237012e-03,1.502150e-03,3.526122e-04,0.234635,82.100246,-82.147526,1650.136169,142.068436
75%,0.249933,0.246463,0.252080,1.882479e-02,1.485101e-02,7.196977e-02,0.251901,84.318287,-71.274647,1666.600311,402.496101
max,0.456576,0.353023,0.362429,3.300374e-01,1.768186e-01,2.284313e-01,0.327535,86.686356,-39.878021,1707.645020,919.184143


In [199]:
X = df.drop(columns=['Epit'])
y = df['Epit']

# Surrogate Model training

In [203]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [204]:
scaler = MinMaxScaler(feature_range=(-1, 1))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

y_scaler = MinMaxScaler(feature_range=(-1, 1))
y_train_scaled = y_scaler.fit_transform(y_train.to_numpy().reshape(-1, 1)).flatten()
y_test_scaled = y_scaler.transform(y_test.to_numpy().reshape(-1, 1)).flatten()

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)

train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
model = RobustNN(input_dim=X_train.shape[1])
model = train_model(model, train_loader, test_loader, l1_lambda=0.02)
preds, trues = evaluate_model(model, test_loader, y_scaler)

# Bayesian Optimization with Elements Composition Only

In [ ]:
ELEMENTS = ['Fe','Cr','Ni','Mn','Al','Cu','Co']
lower = np.array([0.20, 0.20, 0.20, 0.00, 0.02, 0.00, 0.20], dtype=np.float32)
upper = np.array([0.30, 0.35, 0.30, 0.00, 0.06, 0.00, 0.30], dtype=np.float32)

SOFTMAX_T = 0.3
BOX_PENALTY = 5e3        
MN_CU_PENALTY = 5.0       
FEASIBILITY_HARD = 1e6   

def softmax(x, temp=1.0, axis=-1):
    z = (x / temp) - np.max(x / temp, axis=axis, keepdims=True)
    e = np.exp(z)
    return e / e.sum(axis=axis, keepdims=True)

def project_simplex_bounded(w, lo, hi):
    x = w.copy()
    x = np.clip(x, lo, hi)
    s = x.sum()
    if s != 0:
        x = x / s
        x = np.clip(x, lo, hi)
        x = x / x.sum()
    return x

def composition_from_raw(r):
    """
    r: [r_Fe, r_Cr, r_Ni, r_Mn, r_Al, r_Cu, r_Co]
    return：x_comp（sum = 1)，boolean
    """
    w = softmax(np.asarray(r, dtype=np.float32), temp=SOFTMAX_T)
    x = project_simplex_bounded(w, lower, upper)

    if lower.sum() > 1 + 1e-9 or upper.sum() < 1 - 1e-9:
        return x, False  
    return x, True

def black_box_function_dirichlet(r_Fe, r_Cr, r_Ni, r_Mn, r_Al, r_Cu, r_Co):
    r = [r_Fe, r_Cr, r_Ni, r_Mn, r_Al, r_Cu, r_Co]
    x_comp, feas_ok = composition_from_raw(r)
    below = np.maximum(0.0, lower - x_comp)
    above = np.maximum(0.0, x_comp - upper)
    box_violation = below.sum() + above.sum()
    penalty = BOX_PENALTY * box_violation
    if not feas_ok:
        return -FEASIBILITY_HARD
    Mn = x_comp[ELEMENTS.index('Mn')]
    Cu = x_comp[ELEMENTS.index('Cu')]
    penalty += MN_CU_PENALTY * (Mn + Cu)

    x_df = pd.DataFrame([x_comp], columns=ELEMENTS)
    x_scaled = scaler.transform(x_df)
    x_tensor = torch.tensor(x_scaled, dtype=torch.float32)
    
    model.eval()
    with torch.no_grad():
        y_scaled = model(x_tensor).item()
    y_real = y_scaler.inverse_transform([[y_scaled]])[0][0]

    return y_real - penalty

pbounds_raw = {
    'r_Fe': (0.0, 1.0),
    'r_Cr': (0.0, 1.0),
    'r_Ni': (0.0, 1.0),
    'r_Mn': (0.0, 1.0),
    'r_Al': (0.0, 1.0),
    'r_Cu': (0.0, 1.0),
    'r_Co': (0.0, 1.0),
}

optimizer = BayesianOptimization(
    f=black_box_function_dirichlet,
    pbounds=pbounds_raw,
    random_state=42,
    verbose=2
)

In [ ]:
optimizer.maximize(init_points=10, n_iter=25)
best_params = optimizer.max['params']
r_star = [best_params['r_'+e] for e in ELEMENTS]
x_star, _ = composition_from_raw(r_star)

print("\n最优合金元素比例（和=1）：")
for ele, val in zip(ELEMENTS, x_star):
    print(f"{ele}: {val:.4f}")

print(f"\n目标函数（含惩罚）最大值: {optimizer.max['target']:.4f}")

|   iter    |  target   |   r_Fe    |   r_Cr    |   r_Ni    |   r_Mn    |   r_Al    |   r_Cu    |   r_Co    |
-------------------------------------------------------------------------------------------------------------
| 1         | 1098.5967 | 0.3745401 | 0.9507143 | 0.7319939 | 0.5986584 | 0.1560186 | 0.1559945 | 0.0580836 |
| 2         | 765.53136 | 0.8661761 | 0.6011150 | 0.7080725 | 0.0205844 | 0.9699098 | 0.8324426 | 0.2123391 |
| 3         | 635.60043 | 0.1818249 | 0.1834045 | 0.3042422 | 0.5247564 | 0.4319450 | 0.2912291 | 0.6118528 |
| 4         | 767.64367 | 0.1394938 | 0.2921446 | 0.3663618 | 0.4560699 | 0.7851759 | 0.1996737 | 0.5142344 |
| 5         | 581.59655 | 0.5924145 | 0.0464504 | 0.6075448 | 0.1705241 | 0.0650515 | 0.9488855 | 0.9656320 |
| 6         | 710.76810 | 0.8083973 | 0.3046137 | 0.0976721 | 0.6842330 | 0.4401524 | 0.1220382 | 0.4951769 |
| 7         | 1155.9052 | 0.0343885 | 0.9093204 | 0.2587799 | 0.6625222 | 0.3117110 | 0.5200680 | 0.5467102 |
| 8       

# Bayesian Optimization with Additional Calculated Features

In [ ]:
ELEMENTS = ['Fe','Cr','Ni','Mn','Al','Cu','Co']
EXTRA_COLS = ['E_M-M', 'Gibbs free energy of oxide formation_Aver', 'Energy of ionization second_Aver'] 
lower = np.array([0.20, 0.20, 0.20, 0.00, 0.02, 0.00, 0.20], dtype=np.float32)
upper = np.array([0.30, 0.35, 0.30, 0.00, 0.06, 0.00, 0.30], dtype=np.float32)

SOFTMAX_T = 0.3
BOX_PENALTY = 5e3         
MN_CU_PENALTY = 100       
FEASIBILITY_HARD = 1e6    

K_NEIGHBORS = 15
_nn = NearestNeighbors(n_neighbors=K_NEIGHBORS, metric='euclidean')
_nn.fit(X_train[ELEMENTS].to_numpy(dtype=np.float32))

def infer_extras_by_knn(x_comp: np.ndarray) -> np.ndarray:
    dist, idx = _nn.kneighbors(x_comp.reshape(1, -1), return_distance=True)
    w = 1.0 / (dist + 1e-6)        
    w = (w / w.sum()).ravel()
    extras = (X_train.iloc[idx[0]][EXTRA_COLS]
              .to_numpy(dtype=np.float32) * w[:, None]).sum(axis=0)
    return extras

def build_row_from_comp(x_comp: np.ndarray) -> pd.DataFrame:
    extras = infer_extras_by_knn(x_comp)
    row = np.concatenate([x_comp, extras]).reshape(1, -1)
    return pd.DataFrame(row, columns=ELEMENTS + EXTRA_COLS)

def softmax(x, temp=1.0, axis=-1):
    z = (x / temp) - np.max(x / temp, axis=axis, keepdims=True)
    e = np.exp(z)
    return e / e.sum(axis=axis, keepdims=True)

def project_simplex_bounded(w, lo, hi):
    x = w.copy()
    x = np.clip(x, lo, hi)
    s = x.sum()
    if s != 0:
        x = x / s
        x = np.clip(x, lo, hi)
        x = x / x.sum()
    return x

def composition_from_raw(r):
    w = softmax(np.asarray(r, dtype=np.float32), temp=SOFTMAX_T)
    x = project_simplex_bounded(w, lower, upper)
    if lower.sum() > 1 + 1e-9 or upper.sum() < 1 - 1e-9:
        return x, False
    return x, True

def black_box_function_dirichlet(r_Fe, r_Cr, r_Ni, r_Mn, r_Al, r_Cu, r_Co):
    r = [r_Fe, r_Cr, r_Ni, r_Mn, r_Al, r_Cu, r_Co]
    x_comp, feas_ok = composition_from_raw(r)

    below = np.maximum(0.0, lower - x_comp)
    above = np.maximum(0.0, x_comp - upper)
    box_violation = below.sum() + above.sum()
    penalty = BOX_PENALTY * box_violation
    if not feas_ok:
        return -FEASIBILITY_HARD

    Mn = x_comp[ELEMENTS.index('Mn')]
    Cu = x_comp[ELEMENTS.index('Cu')]
    penalty += MN_CU_PENALTY * (Mn + Cu)

    x_df = build_row_from_comp(x_comp)                    
    x_scaled = scaler.transform(x_df)
    x_tensor = torch.tensor(x_scaled, dtype=torch.float32)

    model.eval()
    with torch.no_grad():
        y_scaled = model(x_tensor).item()
    y_real = y_scaler.inverse_transform([[y_scaled]])[0][0]

    return y_real - penalty

pbounds_raw = {
    'r_Fe': (0.0, 1.0),
    'r_Cr': (0.0, 1.0),
    'r_Ni': (0.0, 1.0),
    'r_Mn': (0.0, 1.0),
    'r_Al': (0.0, 1.0),
    'r_Cu': (0.0, 1.0),
    'r_Co': (0.0, 1.0),
}

optimizer = BayesianOptimization(
    f=black_box_function_dirichlet,
    pbounds=pbounds_raw,
    random_state=42,
    verbose=2
)


In [178]:
optimizer.maximize(init_points=10, n_iter=25)
best_params = optimizer.max['params']
r_star = [best_params['r_'+e] for e in ELEMENTS]
x_star, _ = composition_from_raw(r_star)

print("\n最优合金元素比例（归一化后，和=1）：")
for ele, val in zip(ELEMENTS, x_star):
    print(f"{ele}: {val:.4f}")

print(f"\n目标函数（含惩罚）最大值: {optimizer.max['target']:.4f}")


|   iter    |  target   |   r_Fe    |   r_Cr    |   r_Ni    |   r_Mn    |   r_Al    |   r_Cu    |   r_Co    |
-------------------------------------------------------------------------------------------------------------
| 1         | 668.63409 | 0.3745401 | 0.9507143 | 0.7319939 | 0.5986584 | 0.1560186 | 0.1559945 | 0.0580836 |
| 2         | 419.76615 | 0.8661761 | 0.6011150 | 0.7080725 | 0.0205844 | 0.9699098 | 0.8324426 | 0.2123391 |
| 3         | 537.57606 | 0.1818249 | 0.1834045 | 0.3042422 | 0.5247564 | 0.4319450 | 0.2912291 | 0.6118528 |
| 4         | 491.86251 | 0.1394938 | 0.2921446 | 0.3663618 | 0.4560699 | 0.7851759 | 0.1996737 | 0.5142344 |
| 5         | 469.09721 | 0.5924145 | 0.0464504 | 0.6075448 | 0.1705241 | 0.0650515 | 0.9488855 | 0.9656320 |
| 6         | 218.04945 | 0.8083973 | 0.3046137 | 0.0976721 | 0.6842330 | 0.4401524 | 0.1220382 | 0.4951769 |
| 7         | 595.74094 | 0.0343885 | 0.9093204 | 0.2587799 | 0.6625222 | 0.3117110 | 0.5200680 | 0.5467102 |
| 8       

In [167]:
extras_star = infer_extras_by_knn(np.array(x_star))
print("\n对应的三个额外特征（由 kNN 条件均值推断）：")
for col, val in zip(EXTRA_COLS, extras_star):
    print(f"{col}: {val:.4f}")


对应的三个额外特征（由 kNN 条件均值推断）：
E_M-M: 83.9472
Gibbs free energy of oxide formation_Aver: -97.8205
Energy of ionization second_Aver: 1645.3637


# MPEA1 and MPEA2

In [47]:
hea_1_composition = {
    'Fe': 0.2128,
    'Cr': 0.3191,
    'Ni': 0.2127,
    'Mn': 0.00,
    'Al': 0.0426,
    'Cu': 0.00,
    'Co': 0.2127,
}
hea_2_composition = {
    'Fe': 0.202,
    'Cr': 0.3535,
    'Ni': 0.202,
    'Mn': 0.00,
    'Al': 0.0404,
    'Cu': 0.00,
    'Co': 0.2021,
}
hea_1_full = {
    'Fe': 0.2128,
    'Cr': 0.3191,
    'Ni': 0.2127,
    'Mn': 0.00,
    'Al': 0.0426,
    'Cu': 0.00,
    'Co': 0.2127,
    'E_M-M': 85.0737,
    'Gibbs free energy of oxide formation_Aver': -142.11425,
    'Energy of ionization second_Aver': 1640.7241,
}
hea_2_full = {
    'Fe': 0.202,
    'Cr': 0.3535,
    'Ni': 0.202,
    'Mn': 0.00,
    'Al': 0.0404,
    'Cu': 0.00,
    'Co': 0.2021,
    'E_M-M': 85.736856,
    'Gibbs free energy of oxide formation_Aver': -152.15142,
    'Energy of ionization second_Aver': 1638.2741,
}


# Forward Prediction Validation

In [88]:
tmp = pd.DataFrame([hea_1_composition])[X_train.columns]
tmp_scaled = scaler.transform(tmp)
tmp_tensor = torch.tensor(tmp_scaled, dtype=torch.float32)
model.eval()
with torch.no_grad():
    y_scaled = model(tmp_tensor).item()
y_real = y_scaler.inverse_transform([[y_scaled]])[0][0]
print(f"Predicted Epit for HEA-1: {y_real:.4f}")

Predicted Epit for HEA-1: 842.5880


In [101]:
tmp_1 = pd.DataFrame([hea_2_composition])[X_train.columns]
tmp_1_scaled = scaler.transform(tmp_1)
tmp_1_tensor = torch.tensor(tmp_1_scaled, dtype=torch.float32)
model.eval()
with torch.no_grad():
    y_1_scaled = model(tmp_1_tensor).item()
y_1_real = y_scaler.inverse_transform([[y_1_scaled]])[0][0]
print(f"Predicted Epit for HEA-2: {y_1_real:.4f}")

Predicted Epit for HEA-2: 1024.6275


# Forward Prediction Validation

In [175]:
tmp = pd.DataFrame([hea_1_full])[X_train.columns]
tmp_scaled = scaler.transform(tmp)
tmp_tensor = torch.tensor(tmp_scaled, dtype=torch.float32)
model.eval()
with torch.no_grad():
    y_scaled = model(tmp_tensor).item()
y_real = y_scaler.inverse_transform([[y_scaled]])[0][0]
print(f"Predicted Epit for HEA-1: {y_real:.4f}")

Predicted Epit for HEA-1: 745.4340


In [176]:
tmp_1 = pd.DataFrame([hea_2_full])[X_train.columns]
tmp_1_scaled = scaler.transform(tmp_1)
tmp_1_tensor = torch.tensor(tmp_1_scaled, dtype=torch.float32)
model.eval()
with torch.no_grad():
    y_1_scaled = model(tmp_1_tensor).item()
y_1_real = y_scaler.inverse_transform([[y_1_scaled]])[0][0]
print(f"Predicted Epit for HEA-2: {y_1_real:.4f}")

Predicted Epit for HEA-2: 832.5305
